<a href="https://colab.research.google.com/github/marquezjp/SQLToPandas/blob/main/SQLite/Basico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipular SQLite

## Preparar Ambiente

In [ ]:
# Carrega a Biblioteca do Oracle, YAML e Pandas
from datetime import date
import csv
import pandas as pd

In [ ]:
import sqlite3

In [ ]:
from sqlalchemy.engine import create_engine

In [ ]:
today = date.today().strftime("%d/%m/%Y")
print("Hoje é", today)

In [ ]:
!dir

## Conexão com SQLite

In [ ]:
# Cria Paramentros de Configuração do Banco de Dados
sqliteConfig = {
    'dialect': 'sqlite',
    'dbworkpath': 'Dados/SQLite',
    'dbfilename': 'sigrh',
    'dbext': 'sqlite'
}

In [ ]:
# Cria a Conexão Direta no SQLite
dbworkpath = sqliteConfig.get('dbworkpath', '')
dbfilename = sqliteConfig.get('dbfilename', '')
dbext = sqliteConfig.get('dbext', '')
dburl = f'{dbworkpath}/{dbfilename}.{dbext}' if dbfilename else ''

conexaoSQLite = sqlite3.connect(dburl)
conexaoSQLite

In [ ]:
# Cria a Conexão do SQLite via SQL Alchemy
dialeto = sqliteConfig.get('dialect', 'sqlite')

dbworkpath = sqliteConfig.get('dbworkpath', '')
dbfilename = sqliteConfig.get('dbfilename', '')
dbext = sqliteConfig.get('dbext', '')
dburl = f'{dbworkpath}/{dbfilename}.{dbext}' if dbfilename else ''

conexaoDB = create_engine(f'{dialeto}:///{dburl}', echo=False)
conexaoDB

## Consulta SQL

**Principais comandos do SQLite**

**Verificar as Tabelas Existentes**

```
SELECT name FROM sqlite_master WHERE type="table"

SELECT sql FROM sqlite_master WHERE type = "table" AND name = "table_name"

PRAGMA table_info("table_name")

DROP TABLE [IF EXISTS] table_name
```

**Verificar os Index Existentes**

```
SELECT * FROM sqlite_master WHERE type = "index"

SELECT type, name, tbl_name, sql FROM sqlite_master WHERE type= "index"

PRAGMA index_list("table_name")

PRAGMA index_info("index_name")

CREATE INDEX idx_contacts_name ON contacts (first_name, last_name)

CREATE [UNIQUE] INDEX index_name ON table_name ( column_name [, ...] )

DROP INDEX [IF EXISTS] index_name
```

**Exemplo de Create Table e Index**
```
CREATE TABLE [IF NOT EXISTS] table_name(
  chng_id INTEGER PRIMARY KEY,
  acct_no INTEGER REFERENCES account,
  location INTEGER REFERENCES locations,
  amt INTEGER,  -- in cents
  authority TEXT,
  comment TEXT
)

CREATE INDEX index_name ON table_name(acct_no, abs(amt))
```

## Executar Comandos SQLite

In [ ]:
# Excecutar comandos SQLite com conexão direta no SQLite
%%time
cursor = conexaoSQLite.cursor()
cursor.execute('SELECT name FROM sqlite_master WHERE type="table"')
print(cursor.fetchall())
cursor.close()

In [ ]:
# Excecutar comandos SQLite com conexão via SQL Alchemy
%%time
with conexaoDB.connect() as conecta:
    for linha in conecta.execute(f'SELECT name FROM sqlite_master WHERE type="table"'):
        print (linha)

In [ ]:
# Excecutar select do SQLite com conexão via Pandas e SQL Alchemy
%%time
base = pd.read_sql_query('SELECT * FROM folhas', conexaoDB)
base

## Salva os Dados Selecionados no Arquivo CSV

In [ ]:
# Salvar os dados selecionados no arquivo CSV
%%time
nomeArquivo = 'nomeTabela'
base.to_csv(f'Dados/{nomeArquivo}.csv', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
# Salvar os dados selecionados no arquivo ZIP
%%time
nomeArquivo = 'nomeTabela'
compression_opts = dict(method='zip', archive_name=f'{nomeArquivo}.csv')
base.to_csv(f'Dados/{nomeArquivo}.zip', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC, compression=compression_opts)

## Salva os Dados Selecionados no SQLite

In [ ]:
%%time
nomeTabela = 'nomeTabela'
base.to_sql(name=nomeTabela, con=conexaoDB, if_exists="replace", index=False)

In [ ]:
%%time
pd.read_sql(f'select * from {nomeTabela}', conexaoDB).head()

## Salva os Dados Selecionados em partes no SQLite

In [ ]:
%%time
tamanhoPartes=10**3
nomeArquivo = 'nomeTabela'
nomeTabela = 'nomeTabela'
for partesArq in pd.read_csv(f'{nomeArquivo}.csv', encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC, chunksize=tamanhoPartes):
  partesArq.to_sql(name=nomeTabela, con=conexaoDB, if_exists="append", index=False)
  print(partesArq.iloc[0, 1])

In [ ]:
%%time
pd.read_sql(f'select count(*) from {nomeTabela}', conexaoSQLite)

## Mostrar os Nomes de Tabelas e Nomes de Colunas de Todas as Tabelas do SQLite  

In [ ]:
# Mostrar os Nomes de Tabelas e Nomes de Colunas de Todas as Tabelas do SQLite
newline_indent = '\n   '
conexaoSQLite.text_factory = str
cursor = conexaoSQLite.cursor()

result = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = sorted(list(zip(*result))[0])
print ("\nLista de Tabelas:"+newline_indent+newline_indent.join(table_names))

for table_name in table_names:
    result = cursor.execute("PRAGMA table_info('%s')" % table_name).fetchall()
    column_names = list(zip(*result))[1]
    print (("\nNomes das Colunas da Tabela %s:" % table_name)
           +newline_indent
           +(newline_indent.join(column_names)))

cursor.close()

## Gerar DDL de todas as Tabelas de um Banco de Dados SQLite  

In [ ]:
# Gerar DDL de todas as Tabelas de um Banco de Dados SQLite
cursor = conexaoSQLite.cursor()

result = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = sorted(list(zip(*result))[0])

newline_indent = '\n   '
print ("Lista de Tabelas:"+newline_indent+newline_indent.join(table_names))
print('\n')

for table_name in table_names:
    result = cursor.execute("SELECT sql FROM sqlite_master WHERE type = 'table' AND name = '%s'" % table_name).fetchall()
    print ("DDL da Tabela %s:" % table_name)
    for linha in result[0]:
      print (linha)
    print(';\n')

cursor.close()

## Dump de todas as Tabelas de um Banco de Dados SQLite

In [ ]:
# Dump de todas as Tebelas um Banco de Dados SQLite para arquivos CSV 
cursor = conexaoSQLite.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, conexaoSQLite)
    table.to_csv('Dados/Dump/' + table_name + '.csv', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC)
cursor.close()

## Dump de um Banco de Dados SQLite

In [ ]:
# Dump de um Banco de Dados SQLite
nomeArquivoDump = 'Dados/Dump/dump.sql'
with open(nomeArquivoDump, 'w') as f:
    for line in conexaoSQLite.iterdump():
        f.write('%s\n' % line)